### Jupyter Notebook Eksempel – KI-agenter med CrewAI for eHjerteRehab

- A.L. 2025-05-23 (using the `hjerterehab` conda environment defined in big_environment.yml)

- A.L. 2025-05-25 (rename to eHjerteRehab.ipynb) using `ehjerterehab_crew_env` conda environment defined below

```bash
# Create a simple conda environment first
conda create -n ehjerterehab_crew_env python=3.12 -y
conda activate ehjerterehab_crew_env

# Install packages individually with version constraints
pip install "crewai>=0.28.0"
pip install "crewai-tools>=0.4.0" 
pip install "langchain-google-genai>=1.0.0"
pip install "python-dotenv>=1.0.0"
pip install "jupyter>=1.0"

# pip install --upgrade langchain-google-genai
```

**Formål:** Å demonstrere konseptuelt hvordan et team av KI-agenter kan samarbeide om en oppgave relevant for persontilpasset rehabilitering i eHjerteRehab, ved bruk av rammeverket CrewAI. Dette eksempelet er forenklet og bruker simulerte data for illustrative formål.

### CrewAI Demo: Persontilpasset Aktivitetsforslag for eHjerteRehab

**Formål:** Dette Jupyter Notebook-eksempelet demonstrerer hvordan et team av spesialiserte KI-agenter, bygget med CrewAI, kan samarbeide for å analysere en (simulert) pasientprofil og generere persontilpassede aktivitetsforslag med en påfølgende etisk vurdering. Dette illustrerer en potensiell anvendelse av KI for å støtte persontilpasning i eHjerteRehab.

**Scenario:**

1. En `DataFortolkerAgent` analyserer en pasientprofil for å identifisere nøkkelinnsikter.

2. En `AktivitetsDesignerAgent` bruker disse innsiktene til å foreslå rehabiliteringsaktiviteter.

3. En `EtikkOgSamsvarsAgent` vurderer forslagene fra et etisk og ansvarlig perspektiv.

**Merk**: Dette er en konseptuell demonstrasjon og bruker ikke reelle pasientdata eller avanserte medisinske verktøy.

### Hvorfor CrewAI for denne demonstrasjonen?

Valget av CrewAI  for denne demonstrasjonen er strategisk. eHjerteRehab-prosjektet er i seg selv et multimodalt og tverrfaglig initiativ med mange aktører. CrewAI tillater oss å speile denne tverrfaglige samarbeidsmodellen i en KI-arkitektur. 
  

- **Spesialiserte Agenter**: I stedet for en enkelt, monolitisk KI (som kan oppleves som en "svart boks"), lar CrewAI oss definere agenter med distinkte roller, mål og bakgrunnshistorier. Dette kan sammenlignes med hvordan ulike fageksperter (kardiolog, fysioterapeut, psykolog, etiker) samarbeider i helsevesenet.

- **Transparent Prosess**: Ved å observere interaksjonen mellom agentene, blir KI-prosessen potensielt mer transparent og forståelig. Man kan se hvordan ulike "ekspertiser" (simulert av agentene) bidrar til det endelige resultatet.

- **Tilgjengeliggjøring av KI**: En slik agentbasert tilnærming kan gjøre KI-konsepter mer tilgjengelige og mindre truende for helsepersonell og andre interessenter, da det bygger på kjente prinsipper for teamarbeid og ansvarsdeling. Dette er i tråd med eHjerteRehabs mål om å modernisere og digitalisere arbeidsprosesser på en forståelig måte.

### Oppsett av Miljøet

For å kjøre dette eksempelet, trenger du:

1. Python 3.10 eller nyere.

2. CrewAI og nødvendige avhengigheter installert (`pip install crewai crewai-tools`).

3. En API-nøkkel for en stor språkmodell (LLM), f.eks. fra OpenAI, satt som en miljøvariabel (`OPENAI_API_KEY`). For Helse Bergen-kontekst, vil man typisk bruke sikrede, godkjente modeller og plattformer.

Vi starter med å importere nødvendige biblioteker og laste miljøvariabler (hvis du bruker en 
`.env`-fil).

###  Importering og Miljøvariabler

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
# from langchain_openai import ChatOpenAI # Kan aktiveres for spesifikk modellstyring

# Last miljøvariabler fra.env-filen (hvis den finnes)
load_dotenv()

# Sett opp API-nøkkel og eventuelt modellnavn
# Dette hentes typisk fra miljøvariabler for sikkerhetens skyld
# os.environ["OPENAI_API_KEY"] = "DIN_API_NØKKEL_HER" # Bør settes i.env
# os.environ = "gpt-4-turbo" # Eksempel, velg modell etter behov/tilgang

# For demonstrasjonens skyld, hvis OPENAI_API_KEY ikke er satt, kan man
# legge inn en påminnelse eller stoppe kjøringen.
# I et produksjonsmiljø i helsevesenet ville tilgang og autentisering
# vært håndtert via sikre, godkjente systemer.

if not os.getenv("OPENAI_API_KEY"):
    print("ADVARSEL: OPENAI_API_KEY er ikke satt som miljøvariabel.")
    print("Eksempelet vil sannsynligvis feile uten API-tilgang til en LLM.")
    # For en reell kjøring, ville man her kanskje avsluttet eller brukt en fallback.

### Agentdefinisjoner – Vårt KI-Team

Vi definerer nå våre tre KI-agenter. Hver agent får en spesifikk `role` (rolle), et `goal` (mål), og en `backstory` (bakgrunnshistorie). Disse elementene er ikke bare beskrivende; de fungerer som avanserte instrukser (prompts) som former hvordan LLM-en utfører agentens oppgaver. 

En velutformet `backstory` er avgjørende for å instruere LLM-en til å adoptere en spesifikk faglig identitet og tilnærming, noe som direkte påvirker kvaliteten og relevansen av agentens output. Dette er spesielt viktig i spesialiserte domener som helse.   

Vi setter `verbose=True` for å kunne følge agentenes "tankeprosess" under kjøring. `allow_delegation=False` betyr at agentene ikke kan delegere oppgaver til andre agenter i dette enkle scenarioet.

### Agent 1 - DataFortolkerAgent

Denne agenten vil 'lese' pasientprofilen og trekke ut det viktigste for tilpasning, med referanser til relevante arbeidspakker i eHjerteRehab.

In [3]:
data_fortolker_agent = Agent(
    role='Senior HelseData Analytiker for Hjerterehabilitering',
    goal=('Analysere en simulert pasientprofil for å identifisere 3-5 nøkkelinnsikter som er kritiske '
          'for å skreddersy en digital rehabiliteringsplan. Fokusere på motivasjonsfaktorer, '
          'potensielle barrierer, og individuelle preferanser.'),
    backstory=(
        "Du er en erfaren dataanalytiker med doktorgrad i medisinsk informatikk, spesialisert på å "
        "utlede handlingsrettet innsikt fra pasientdata innen kardiologi. Din ekspertise ligger i å "
        "identifisere subtile mønstre som kan forbedre persontilpasset behandling og pasientengasjement i "
        "digitale helseprogrammer som eHjerteRehab. Du er grundig, nøyaktig og opptatt av personvern, "
        "og du forstår viktigheten av å vurdere faktorer som komorbiditet (jf. eHjerteRehab WP1.3) "
        "og mental helse (jf. eHjerteRehab WP1.4) i din analyse.[1]"
    ),
    verbose=True,
    allow_delegation=False
)

### Agent 2 - AktivitetsDesignerAgent

Denne agenten tar innsiktene og omgjør dem til konkrete aktivitetsforslag, med tanke på engasjement og helsekompetanse.

In [4]:
aktivitets_designer_agent = Agent(
    role='Innovativ Rehabiliteringsspesialist med fokus på Digitalt Engasjement',
    goal=('Basert på innsiktene fra DataFortolkerAgent, utvikle 2-3 konkrete, kreative og motiverende '
          'forslag til persontilpassede rehabiliteringsaktiviteter for eHjerteRehab. Aktivitetene skal '
          'være gjennomførbare hjemme og tilpasset pasientens antatte fysiske og mentale kapasitet.'),
    backstory=(
        "Du er en fysioterapeut og rehabiliteringsforsker med en lidenskap for å integrere teknologi "
        "og spillifisering i helsefremmende arbeid. Du har jobbet tett med utviklingen av eHjerteRehab "
        "og forstår viktigheten av å gjøre rehabilitering både effektiv og engasjerende, samtidig som "
        "du tar hensyn til pasientens helsekompetanse (jf. eHjerteRehab WP2.2).[1] Ditt mål er "
        "å 'tenne gnisten' hos pasienten gjennom skreddersydde og motiverende aktiviteter."
    ),
    verbose=True,
    allow_delegation=False
)

### Agent 3 - EtikkOgSamsvarsAgent

Denne agenten fungerer som en 'etisk vokter' for å sikre ansvarlig praksis, i tråd med eHjerteRehabs WP2.5 Etikk

In [5]:
etikk_og_samsvars_agent = Agent(
    role='Ekspert på Medisinsk Etikk og Ansvarlig KI i Helsevesenet',
    goal=('Kritisk vurdere de foreslåtte rehabiliteringsaktivitetene fra AktivitetsDesignerAgent. '
          'Sikre at forslagene er i tråd med etiske retningslinjer for eHjerteRehab (jf. WP2.5 Etikk [1]), '
          'respekterer pasientautonomi, unngår utilsiktet bias, og ivaretar personvern. '
          'Gi konkret tilbakemelding og eventuelle justeringsforslag.'),
    backstory=(
        "Du er en etiker med juridisk bakgrunn og spesialkompetanse på GDPR og KI-regulering i helsesektoren. "
        "Du er tilknyttet eHjerteRehabs ekspertgruppe og Scientific Advisory Board [1] "
        "og har som hovedoppgave å sikre at alle digitale intervensjoner er "
        "etisk forsvarlige, rettferdige og trygge for pasientene. Du er grundig og prinsippfast, "
        "men også løsningsorientert, og legger vekt på kontinuitet i helsetjenesten (jf. eHjerteRehab WP2.1).[1]"
    ),
    verbose=True,
    allow_delegation=False
)

### Oppgavedefinisjoner – Hva Skal Agentene Gjøre?

Nå definerer vi oppgavene (Task) som agentene skal utføre. Hver oppgave har en description (beskrivelse av hva som skal gjøres), en expected_output (forventet format og innhold på resultatet), og tildeles en agent.   

Å definere expected_output tydelig er kritisk. Det styrer agentens atferd og sikrer at outputen er strukturert og anvendelig for påfølgende oppgaver eller for menneskelig tolkning. Dette reduserer "støy" og øker effektiviteten i agent-samarbeidet.

Vi vil bruke en fiktiv, men relevant, pasientprofil som input til den første oppgaven.


### Fiktiv Pasientprofil for "Ola Nordmann" (67 år)

- Medisinsk Historikk: Gjennomgått PCI for 3 uker siden etter et hjerteinfarkt. Har også kjent hypertensjon og diabetes type 2 (moderat regulert). Lett overvektig.

- Livsstil: Stillesittende kontorjobb før pensjonsalder (nylig pensjonert). Røyket tidligere, men sluttet for 5 år siden. Moderat alkoholforbruk i helgene. Bor alene i enebolig med hage, et stykke utenfor sentrum.

- Psykososialt: Uttrykker bekymring for fremtiden og er redd for nytt infarkt. Føler seg litt isolert etter pensjonering. Motivasjon for trening er "så som så", men han liker å være ute i naturen og "pusle med ting". Teknologisk kompetanse: Bruker smarttelefon og nettbrett til enkle ting (nyheter, e-post), men er ikke veldig dreven.

- Rehabiliteringsmål (fra samtale): Ønsker å "komme i bedre form", "føle seg tryggere", og "ha noe meningsfylt å gjøre".

Denne profilen gir nok detaljer til at agentene kan jobbe med den. I eHjerteRehab vil reelle data være mer omfattende og dynamiske.   



### Oppgave 1 - AnalyserPasientprofil


In [6]:
pasientprofil_tekst = """
Medisinsk Historikk: Gjennomgått PCI for 3 uker siden etter et hjerteinfarkt. Har også kjent hypertensjon og diabetes type 2 (moderat regulert). Lett overvektig.
Livsstil: Stillesittende kontorjobb før pensjonsalder (nylig pensjonert). Røyket tidligere, men sluttet for 5 år siden. Moderat alkoholforbruk i helgene. Bor alene i enebolig med hage, et stykke utenfor sentrum.
Psykososialt: Uttrykker bekymring for fremtiden og er redd for nytt infarkt. Føler seg litt isolert etter pensjonering. Motivasjon for trening er "så som så", men han liker å være ute i naturen og "pusle med ting". Teknologisk kompetanse: Bruker smarttelefon og nettbrett til enkle ting (nyheter, e-post), men er ikke veldig dreven.
Rehabiliteringsmål (fra samtale): Ønsker å "komme i bedre form", "føle seg tryggere", og "ha noe meningsfylt å gjøre".
"""

task_analyser_profil = Task(
    description=(f"Analyser følgende simulerte pasientprofil for 'Ola Nordmann' (67 år):\n\n{pasientprofil_tekst}\n\n"
                 "Identifiser 3-5 nøkkelinnsikter som er avgjørende for å persontilpasse hans eHjerteRehab-program. "
                 "Fokuser på motivasjonsdrivere, potensielle barrierer (fysiske, psykososiale, teknologiske), "
                 "og hans uttrykte mål og preferanser. Vurder spesielt hans komorbiditeter (hypertensjon, diabetes) "
                 "og hvordan disse kan påvirke rehabiliteringsplanen, i tråd med fokus i eHjerteRehab WP1.3.[1]"),
    expected_output=("En punktliste med 3-5 klare og konsise nøkkelinnsikter, hver med en kort begrunnelse. "
                     "Innsiktene skal være direkte anvendbare for neste agent som skal designe aktiviteter."),
    agent=data_fortolker_agent
)

### Oppgave 2 - UtformAktivitetsforslag

Bruken av `context=[task_analyser_profil]`  er fundamental. Det skaper en logisk arbeidsflyt der `AktivitetsDesignerAgent` bygger direkte på arbeidet til `DataFortolkerAgent`. Dette simulerer informasjonsdeling i et team og er kjernen i hvordan CrewAI muliggjør samarbeid.

In [7]:
task_utform_aktiviteter = Task(
    description=("Basert på nøkkelinnsiktene fra analysen av Ola Nordmanns pasientprofil (levert som kontekst), "
                 "utvikle 2-3 konkrete, persontilpassede og motiverende aktivitetselementer for hans eHjerteRehab-program. "
                 "Aktivitetene bør være trygge med tanke på hans medisinske historikk (PCI, hypertensjon, diabetes). "
                 "Inkluder en kort begrunnelse for hvert forslag som knytter det til pasientens profil (f.eks. hans interesse for natur, "
                 "behov for trygghet, teknologiske ferdigheter – husk eHjerteRehabs fokus på brukervennlighet og tilpasset informasjon [1])."),
    expected_output=("En liste med 2-3 detaljerte aktivitetselementer. Hvert element skal inneholde: \n"
                     "1. Navn på aktiviteten.\n"
                     "2. En kort, engasjerende beskrivelse av aktiviteten.\n"
                     "3. En begrunnelse for hvorfor denne aktiviteten passer for Ola, med referanse til hans profil.\n"
                     "4. Eventuelle forslag til enkel teknologisk støtte (f.eks. påminnelser på telefon, enkel app-funksjon "
                     "som er i tråd med eHjerteRehabs teknologiske plattform [1])."),
    agent=aktivitets_designer_agent,
    context=[task_analyser_profil]  # Output fra forrige task blir input her
)


###  Oppgave 3 - EtiskVurderingAvForslag

In [8]:
task_etisk_vurdering = Task(
    description=("Gjennomgå de foreslåtte rehabiliteringsaktivitetene for Ola Nordmann (levert som kontekst). "
                 "Vurder dem kritisk i lys av prinsipper for ansvarlig KI og medisinsk etikk, spesielt med tanke på: \n"
                 "- Pasientautonomi og informert samtykke (selv om dette er simulert).\n"
                 "- Potensial for utilsiktet bias eller diskriminering (jf. eHjerteRehabs mål om å nå underrepresenterte grupper [1]).\n"
                 "- Personvernsaspekter ved datainnsamling knyttet til aktivitetene (eHjerteRehab bruker BankID og sikker kommunikasjon [1]).\n"
                 "- Rettferdighet og likeverdighet i tilbudet.\n"
                 "- Sikkerhet og potensiell risiko for pasienten (fysisk og psykisk).\n"
                 "Dette arbeidet speiler intensjonen bak eHjerteRehabs WP2.5 Etikk.[1] "
                 "Gi en konkret tilbakemelding. Hvis det er etiske betenkeligheter, foreslå justeringer eller forbehold."),
    expected_output=("En strukturert rapport som inneholder:\n"
                     "1. En overordnet etisk vurdering av aktivitetsforslagene.\n"
                     "2. Identifisering av eventuelle etiske dilemmaer eller risikofaktorer.\n"
                     "3. Konkrete anbefalinger for å styrke den etiske forsvarligheten av forslagene, om nødvendig.\n"
                     "4. En konklusjon om hvorvidt forslagene, med eventuelle justeringer, er etisk akseptable for "
                     "bruk i en eHjerteRehab-kontekst."),
    agent=etikk_og_samsvars_agent,
    context=[task_utform_aktiviteter]
)

###  Sette Sammen og Kjøre Crewet

Nå setter vi sammen agentene og oppgavene i et `Crew`. Et `Crew` representerer den operasjonelle enheten som orkestrerer agentenes arbeid.
Vi spesifiserer `process=Process.sequential`, som betyr at oppgavene utføres i den rekkefølgen de er listet.
Deretter starter vi prosessen med `kickoff()`.   

Innstillingen `verbose=True` er spesielt nyttig under utvikling og for demonstrasjoner som denne. Den gir detaljert innsyn i agentenes "indre liv" – deres resonnement, hvilke (potensielle) verktøy de vurderer, og hvordan de tolker oppgavene sine. Denne transparensen er verdifull for feilsøking, forbedring av agentdesign, og for å bygge forståelse for hvordan systemet fungerer. For et publikum bestående av helsepersonell og forskere, kan det å vise utdrag av slik verbose output være svært pedagogisk for å avmystifisere KI.

### Definere og Kjøre Crewet

In [9]:
# Definere crewet med agentene og oppgavene
eHjerteRehab_crew = Crew(
    agents=[data_fortolker_agent, aktivitets_designer_agent, etikk_og_samsvars_agent],
    tasks=[task_analyser_profil, task_utform_aktiviteter, task_etisk_vurdering],
    process=Process.sequential,  # Oppgavene utføres i rekkefølge
    verbose=False  # True or 2 Gir detaljert output om agentenes tankeprosess og handlinger
)

# Starte crewets arbeid
# Inputs kan gis her hvis den første oppgaven trenger ekstern input
# som ikke er hardkodet i beskrivelsen. I vårt tilfelle er
# pasientprofilen allerede en del av task_analyser_profil.description.
print("Starter eHjerteRehab Crew...\n")
resultat = eHjerteRehab_crew.kickoff()
print("\nCrew-arbeid fullført.")

Starter eHjerteRehab Crew...

# Agent: Senior HelseData Analytiker for Hjerterehabilitering
## Task: Analyser følgende simulerte pasientprofil for 'Ola Nordmann' (67 år):


Medisinsk Historikk: Gjennomgått PCI for 3 uker siden etter et hjerteinfarkt. Har også kjent hypertensjon og diabetes type 2 (moderat regulert). Lett overvektig.
Livsstil: Stillesittende kontorjobb før pensjonsalder (nylig pensjonert). Røyket tidligere, men sluttet for 5 år siden. Moderat alkoholforbruk i helgene. Bor alene i enebolig med hage, et stykke utenfor sentrum.
Psykososialt: Uttrykker bekymring for fremtiden og er redd for nytt infarkt. Føler seg litt isolert etter pensjonering. Motivasjon for trening er "så som så", men han liker å være ute i naturen og "pusle med ting". Teknologisk kompetanse: Bruker smarttelefon og nettbrett til enkle ting (nyheter, e-post), men er ikke veldig dreven.
Rehabiliteringsmål (fra samtale): Ønsker å "komme i bedre form", "føle seg tryggere", og "ha noe meningsfylt å gjøre".



----

### Crew-arbeid:

Starter eHjerteRehab Crew...

### Agent: Senior HelseData Analytiker for Hjerterehabilitering
#### Task: Analyser følgende simulerte pasientprofil for 'Ola Nordmann' (67 år):


Medisinsk Historikk: Gjennomgått PCI for 3 uker siden etter et hjerteinfarkt. Har også kjent hypertensjon og diabetes type 2 (moderat regulert). Lett overvektig.
Livsstil: Stillesittende kontorjobb før pensjonsalder (nylig pensjonert). Røyket tidligere, men sluttet for 5 år siden. Moderat alkoholforbruk i helgene. Bor alene i enebolig med hage, et stykke utenfor sentrum.
Psykososialt: Uttrykker bekymring for fremtiden og er redd for nytt infarkt. Føler seg litt isolert etter pensjonering. Motivasjon for trening er "så som så", men han liker å være ute i naturen og "pusle med ting". Teknologisk kompetanse: Bruker smarttelefon og nettbrett til enkle ting (nyheter, e-post), men er ikke veldig dreven.
Rehabiliteringsmål (fra samtale): Ønsker å "komme i bedre form", "føle seg tryggere", og "ha noe meningsfylt å gjøre".


Identifiser 3-5 nøkkelinnsikter som er avgjørende for å persontilpasse hans eHjerteRehab-program. Fokuser på motivasjonsdrivere, potensielle barrierer (fysiske, psykososiale, teknologiske), og hans uttrykte mål og preferanser. Vurder spesielt hans komorbiditeter (hypertensjon, diabetes) og hvordan disse kan påvirke rehabiliteringsplanen, i tråd med fokus i eHjerteRehab WP1.3.[1]


### Agent: Senior HelseData Analytiker for Hjerterehabilitering
#### Final Answer: 
1. **Motivasjonsdrivende elementer: Naturbasert aktivitet**  
   Ola uttrykker en glede for å være ute i naturen og "pusle med ting". Dette kan utnyttes ved å inkludere uteaktiviteter i rehabiliteringsprogrammet, som gåturer i naturen, hagearbeid, og aktiviteter som kombinerer bevegelse med hans interesse for friluftsliv. Dette vil øke hans motivasjon for å delta.

2. **Psykososiale barrierer: Isolasjon og bekymring**  
   Ola føler seg isolert etter pensjonering og har uttrykt bekymring for sin helse. Det er viktig å implementere et sosialt element i rehabiliteringsprogrammet, som gruppeaktiviteter eller online støttegrupper, for å bygge fellesskap og redusere følelser av isolasjon. Å inkludere elementer av psykososial støtte kan bidra til bedre mental helse og motivasjon.

3. **Teknologisk kompetanse: Enkel digital interaksjon**  
   Selv om Ola har noe teknologisk kompetanse, bør rehabiliteringsprogrammet designes med brukervennlige digital løsninger. Enkle apper eller programvare som står i stil med hans ferdigheter kan introduseres for å holde oversikt over fremgang, skape daglige påminnelser, og gi data om helsen hans. Personlig tilpasset opplæring kan være nyttig for å fjerne teknologiske barrierer.

4. **Kommorbiditeter: Hjertehelsen må prioriteres**  
   Med hans historie av hypertensjon og diabetes type 2, er det avgjørende å utforme treningstiltak som er tilpasset hans helsetilstand. Dette kan inkludere overvåking av hjertefrekvens og blodsukkernivå under aktivitetene for å sikre sikkerhet. Inkludering av ernæringsveiledning tilpasset hans diabetes vil også styrke rehabiliteringsresultatene.

5. **Klarhet i rehabiliteringsmål: Meningsfulle aktiviteter**  
   Ola ønsker "noe meningsfylt å gjøre". Å integrere målorienterte aktiviteter som gir ham en følelse av fremgang og prestasjon, som små personlige utfordringer eller prosjekter (f.eks. hagearbeid, bygge ting), kan øke engasjementet hans. Dette vil bidra til at han føler seg mer trygg og i bedre form, noe som er i tråd med hans rehabiliteringsmål.


### Agent: Innovativ Rehabiliteringsspesialist med fokus på Digitalt Engasjement
#### Task: Basert på nøkkelinnsiktene fra analysen av Ola Nordmanns pasientprofil (levert som kontekst), utvikle 2-3 konkrete, persontilpassede og motiverende aktivitetselementer for hans eHjerteRehab-program. Aktivitetene bør være trygge med tanke på hans medisinske historikk (PCI, hypertensjon, diabetes). Inkluder en kort begrunnelse for hvert forslag som knytter det til pasientens profil (f.eks. hans interesse for natur, behov for trygghet, teknologiske ferdigheter – husk eHjerteRehabs fokus på brukervennlighet og tilpasset informasjon [1]).


### Agent: Innovativ Rehabiliteringsspesialist med fokus på Digitalt Engasjement
#### Final Answer: 
1. **Navn på aktiviteten:** Natur- og hageprosjektet  
   **Beskrivelse:** Ola skal delta i et ukentlig hageprosjekt, der han får mulighet til å plante, stelle og observere vekst av ulike planter eller grønnsaker. Dette kan enkelt startes med potteplanter eller et lite hagebed hjemme, som han kan jobbe med over tid. Aktiviteten inkluderer lette øvelser som å bøye seg, stå og gå, som vil styrke hans fysiske helse i tillegg til å gi en meningsfull opplevelse.  
   **Begrunnelse:** Aktiviteten utnytter Olas glede for natur og hans interesse for å "pusle med ting". Hagearbeid kan også bidra til å redusere følelser av isolasjon og gir ham en aktiv rolle, samtidig som han kan se fremgang og resultater av arbeidet sitt.  
   **Teknologisk støtte:** Ola kan bruke en enkel app for hageplanlegging, med påminnelser for vanning og stell av plantene. I tillegg kan han dokumentere veksten gjennom bilder, noe som kan gi ham visuell tilbakemelding på fremgangen.

2. **Navn på aktiviteten:** Gåtur med venn  
   **Beskrivelse:** Ola oppfordres til å ta med seg en venn eller familiemedlem på ukentlige gåturer i nærområdet, gjerne i en park eller naturområde han liker. Disse turene skal vare minst 30 minutter, og kan kombineres med samtaler og sosialisering underveis.  
   **Begrunnelse:** Dette tiltaket bygger på å redusere Olas følelser av isolasjon ved å inkludere sosial interaksjon, som er viktig i hans rehabiliteringsprosess. Gåturer i naturen vil ikke bare booste hans fysiske aktivitet, men også gi ham psykologiske fordeler fra sosialt samvær.  
   **Teknologisk støtte:** Ola kan bruke en app for å loggføre sine gåturer, sette mål for antall dager han går i uken og få oppdateringer om hans fysiske aktivitet slik at han kan holde fremgangen sin i fokus.

3. **Navn på aktiviteten:** Digitalt støttegruppeprogram  
   **Beskrivelse:** Ola blir tilbudt å delta i en digital støttegruppe via videochat, hvor han kan dele erfaringer, utfordringer og oppnåelser relatert til sin rehabilitering med andre som har lignende helsetilstander. Gruppen vil ha en fasilitator som guider diskusjoner og aktiviteter.  
   **Begrunnelse:** Dette tiltaket adresserer Olas bekymringer knyttet til isolasjon og sikrer at han har en plattform der han kan dele sine tanker og lære av andre. Det vil være en kilde til emosjonell støtte, og samtidig huske på viktigheten av psykososial helse i rehabilitering.  
   **Teknologisk støtte:** Bruk av en enkel videokonferanseapp med påminnelser og muligheten for å chatte i grupper vil sørge for brukervennlighet og lett tilgang til møter, slik at Ola ikke føler seg overveldet av teknologien. 

Disse aktivitetene vil bidra til å holde Ola motivert, engasjert og gi ham en følelse av mening og fremgang, i tråd med hans individuelle rehabiliteringsmål.


### Agent: Ekspert på Medisinsk Etikk og Ansvarlig KI i Helsevesenet
#### Task: Gjennomgå de foreslåtte rehabiliteringsaktivitetene for Ola Nordmann (levert som kontekst). Vurder dem kritisk i lys av prinsipper for ansvarlig KI og medisinsk etikk, spesielt med tanke på: 
- Pasientautonomi og informert samtykke (selv om dette er simulert).
- Potensial for utilsiktet bias eller diskriminering (jf. eHjerteRehabs mål om å nå underrepresenterte grupper [1]).
- Personvernsaspekter ved datainnsamling knyttet til aktivitetene (eHjerteRehab bruker BankID og sikker kommunikasjon [1]).
- Rettferdighet og likeverdighet i tilbudet.
- Sikkerhet og potensiell risiko for pasienten (fysisk og psykisk).
Dette arbeidet speiler intensjonen bak eHjerteRehabs WP2.5 Etikk.[1] Gi en konkret tilbakemelding. Hvis det er etiske betenkeligheter, foreslå justeringer eller forbehold.


### Agent: Ekspert på Medisinsk Etikk og Ansvarlig KI i Helsevesenet
#### Final Answer: 
**1. En overordnet etisk vurdering av aktivitetsforslagene:**  
De foreslåtte rehabiliteringsaktivitetene for Ola Nordmann, inkludert Natur- og hageprosjektet, Gåtur med venn, og Digitalt støttegruppeprogram, fremstår som etisk forsvarlige i sin tilnærming til rehabilitering. Disse aktivitetene er designet for å støtte Olas fysisk og psykisk velvære, og respekterer pasientautonomi ved å legge til rette for valg og samarbeid i rehabiliteringsprosessen. Bruken av teknologi for å fremme disse aktivitetene er godt gjennomtenkt, men det er viktig å sikre at den ikke skaper unødig kompliserte opplevelser som kan føre til angst eller motstand mot bruk.

**2. Identifisering av eventuelle etiske dilemmaer eller risikofaktorer:**  
- **Pasientautonomi og informert samtykke:** Det er essensielt at Ola har mulighet til å gi informert samtykke til deltakelse i digitale plattformer og aktiviteter, samt mulighet til å trekke seg fra disse om ønskelig. Det må være en klar kommunikasjon om hvordan data vil bli brukt og oppbevart.
- **Potensial for utilsiktet bias eller diskriminering:** Det bør vurderes hvordan disse aktivitetene kan tilpasses for å inkludere underrepresenterte grupper, for eksempel personer med nedsatt mobilitet eller de som er mindre komfortable med teknologi. Hvis aktivitetsforslagene er for fokuserte på fysisk aktivitet, kan det risikoen for å ekskludere personer med spesifikke behov.
- **Personvernsaspekter:** Bruken av data gjennom apper for loggføring og video-deltakelse krever strenge sikkerhetstiltak for å beskytte Olas personopplysninger. Det er nødvendig å sikre at all datainnsamling er i samsvar med GDPR-retningslinjene.
- **Rettferdighet og likeverdighet:** Aktivitetene må være tilgjengelige for alle pasienter uavhengig av teknologisk ferdighet eller fysisk kapasitet. Det må legges til rette for at Ola, uavhengig av sine begrensninger, kan delta fullverdig.
- **Sikkerhet og potensiell risiko:** Det er viktig at aktivitetene er tilpasset Olas fysiske helse og ferdigheter for å unngå skader. I tillegg må den psykiske belastningen ved deltakelse i videokonftamtens grupper vurderes.

**3. Konkrete anbefalinger for å styrke den etiske forsvarligheten av forslagene, om nødvendig:**  
- For å styrke pasientautonomi, bør det være en tydelig prosedyre for å dokumentere at Ola har gitt sitt samtykke til deltakelse i aktivitetene, inkludert databruk. 
- Det kan vurderes å tilby alternative aktiviteter for de som har utfordringer med tilgang til teknologi, for eksempel fysiske møter i trygge miljøer.
- For å sikre at ingen pasienter blir diskriminert eller ekskludert, kan man inkludere tilpasninger og alternative aktiviteter i aktivitetsplanen som tar hensyn til forskjellige ferdigheter og preferanser.
- Regelmessig evaluering av aktivitetene bør implementeres for å tilpasse dem etter pasientens respons og behov.

**4. En konklusjon om hvorvidt forslagene, med eventuelle justeringer, er etisk akseptable for bruk i en eHjerteRehab-kontekst:**  
Etter en grundig vurdering er de foreslåtte rehabiliteringsaktiviteter etisk akseptable, forutsatt at de foreslåtte justeringene blir implementert. Med fokus på pasientens autonomi, beskyttelse mot diskriminering, strenge personvernprosedyrer og en forpliktelse til rettferdighet og sikkerhet, kan disse aktivitetene gi Ola en meningsfull og trygg rehabiliteringsreise. Videre bør tiltak for kontinuerlig sporbarhet og tilbakemeldinger fra Ola understøtte muligheten til å justere tiltakene fortløpende, i tråd med hans opplevelser og preferanser.



#### Crew-arbeid fullført.

-----

### Vise Resultatet

Crewet har nå fullført sine oppgaver. Variabelen `resultat` inneholder det endelige outputtet fra den siste oppgaven i sekvensen, som i dette tilfellet er den etiske vurderingen fra `EtikkOgSamsvarsAgent`. Vi skriver ut dette resultatet.

### Skrive ut Resultatet

Output fra `verbose=True` vil ha vist detaljer fra hver agent underveis i kjøringen. Det som skrives ut over er det konsoliderte sluttresultatet fra den siste agenten i kjeden.

In [11]:
print("\n\n##################################################")
print("##### Endelig Resultat fra EtikkOgSamsvarsAgent #####")
print("##################################################\n")
print(resultat)



##################################################
##### Endelig Resultat fra EtikkOgSamsvarsAgent #####
##################################################

**1. Overordnet etisk vurdering av aktivitetsforslagene:**

De foreslåtte rehabiliteringsaktivitetene for Ola Nordmann, inkludert "Naturlig bevegelse", "Hagearbeid for helse", og "Sosialt treningsfellesskap", fremstår som etisk forsvarlige og tilpasset hans interesser samt behov. Aktivitetene fokuserer på å fremme hans fysiske helse, mentale velvære og sosiale interaksjon, som er vesentlige aspekter ved rehabilitering. 

Alle tre aktivitetene er i tråd med prinsippene for pasientautonomi og informert samtykke, da de tar hensyn til Olas personlige interesser og preferanser. Samtidig vil den teknologiske støtten (apper og videomøter) bidra til å øke hans engasjement og mestring av enhver aktivitet, og dermed også fremme hans autonomi.

**2. Identifisering av etiske dilemmaer eller risikofaktorer:**

- **Pasientautonomi og informert

**1. En overordnet etisk vurdering av aktivitetsforslagene:**  
De foreslåtte rehabiliteringsaktivitetene for Ola Nordmann, inkludert Natur- og hageprosjektet, Gåtur med venn, og Digitalt støttegruppeprogram, fremstår som etisk forsvarlige i sin tilnærming til rehabilitering. Disse aktivitetene er designet for å støtte Olas fysisk og psykisk velvære, og respekterer pasientautonomi ved å legge til rette for valg og samarbeid i rehabiliteringsprosessen. Bruken av teknologi for å fremme disse aktivitetene er godt gjennomtenkt, men det er viktig å sikre at den ikke skaper unødig kompliserte opplevelser som kan føre til angst eller motstand mot bruk.

**2. Identifisering av eventuelle etiske dilemmaer eller risikofaktorer:**  
- **Pasientautonomi og informert samtykke:** Det er essensielt at Ola har mulighet til å gi informert samtykke til deltakelse i digitale plattformer og aktiviteter, samt mulighet til å trekke seg fra disse om ønskelig. Det må være en klar kommunikasjon om hvordan data vil bli brukt og oppbevart.
- **Potensial for utilsiktet bias eller diskriminering:** Det bør vurderes hvordan disse aktivitetene kan tilpasses for å inkludere underrepresenterte grupper, for eksempel personer med nedsatt mobilitet eller de som er mindre komfortable med teknologi. Hvis aktivitetsforslagene er for fokuserte på fysisk aktivitet, kan det risikoen for å ekskludere personer med spesifikke behov.
- **Personvernsaspekter:** Bruken av data gjennom apper for loggføring og video-deltakelse krever strenge sikkerhetstiltak for å beskytte Olas personopplysninger. Det er nødvendig å sikre at all datainnsamling er i samsvar med GDPR-retningslinjene.
- **Rettferdighet og likeverdighet:** Aktivitetene må være tilgjengelige for alle pasienter uavhengig av teknologisk ferdighet eller fysisk kapasitet. Det må legges til rette for at Ola, uavhengig av sine begrensninger, kan delta fullverdig.
- **Sikkerhet og potensiell risiko:** Det er viktig at aktivitetene er tilpasset Olas fysiske helse og ferdigheter for å unngå skader. I tillegg må den psykiske belastningen ved deltakelse i videokonftamtens grupper vurderes.

**3. Konkrete anbefalinger for å styrke den etiske forsvarligheten av forslagene, om nødvendig:**  
- For å styrke pasientautonomi, bør det være en tydelig prosedyre for å dokumentere at Ola har gitt sitt samtykke til deltakelse i aktivitetene, inkludert databruk. 
- Det kan vurderes å tilby alternative aktiviteter for de som har utfordringer med tilgang til teknologi, for eksempel fysiske møter i trygge miljøer.
- For å sikre at ingen pasienter blir diskriminert eller ekskludert, kan man inkludere tilpasninger og alternative aktiviteter i aktivitetsplanen som tar hensyn til forskjellige ferdigheter og preferanser.
- Regelmessig evaluering av aktivitetene bør implementeres for å tilpasse dem etter pasientens respons og behov.

**4. En konklusjon om hvorvidt forslagene, med eventuelle justeringer, er etisk akseptable for bruk i en eHjerteRehab-kontekst:**  
Etter en grundig vurdering er de foreslåtte rehabiliteringsaktiviteter etisk akseptable, forutsatt at de foreslåtte justeringene blir implementert. Med fokus på pasientens autonomi, beskyttelse mot diskriminering, strenge personvernprosedyrer og en forpliktelse til rettferdighet og sikkerhet, kan disse aktivitetene gi Ola en meningsfull og trygg rehabiliteringsreise. Videre bør tiltak for kontinuerlig sporbarhet og tilbakemeldinger fra Ola understøtte muligheten til å justere tiltakene fortløpende, i tråd med hans opplevelser og preferanser.

### Tolkning av Resultatene (Eksempel på Refleksjon)

Når man gjennomgår `resultat` fra `EtikkOgSamsvarsAgent` (og den verbose loggen fra kjøringen), bør man vurdere følgende:

- **DataFortolkerAgent**:
   - Hvordan tolket agenten pasientprofilen til "Ola Nordmann"?
   - Var de identifiserte nøkkelinnsiktene relevante og dekkende for hans situasjon (f.eks. isolasjon, naturinteresse, teknologisk kompetanse, komorbiditeter)?

- **AktivitetsDesignerAgent**:
   - Hvilke konkrete aktiviteter ble foreslått?
   - Virker forslagene genuint persontilpassede og motiverende basert på Olas profil?
   - Er de praktisk gjennomførbare hjemme og teknologisk tilpasset hans nivå?

- **EtikkOgSamsvarsAgent**:
   - Hva var den overordnede etiske konklusjonen?
   - Ble det identifisert noen spesifikke etiske dilemmaer eller risikofaktorer (f.eks. knyttet til hans sårbarhet, personvern, eller rettferdighet)?
   - Var anbefalingene for justeringer (hvis noen) fornuftige og gjennomførbare?

Dette forenklede eksempelet, selv med simulerte data, illustrerer potensialet for KI-støttet persontilpasning og hvordan en strukturert tilnærming til ansvarlig praksis kan integreres. Slike systemer kan potensielt bidra til flere av eHjerteRehabs sentrale mål, som å tilby "mer sammenhengende og skreddersydde pasientforløp", styrke "pasienten si evne til å ta styring over eiga helse", og bidra til å "redusera variasjon i kvalitet og tilgjengelegheit til hjarterehabilitering". 

### Begrensninger ved Eksempelet og Veien Videre

Det er viktig å anerkjenne begrensningene ved denne demonstrasjonen. En åpen diskusjon om disse er avgjørende for å bygge troverdighet og sette realistiske forventninger, spesielt overfor et fagkyndig publikum. Dette viser en moden forståelse for teknologiens nåværende kapabiliteter og utfordringer.

**Begrensninger:**

- **Fiktiv og Statisk Data**: Eksempelet bruker en kort, fiktiv pasientprofil. Reelle pasientdata er langt mer komplekse, dynamiske, og sensitive.

- **Mangel på Reelle Verktøy**: Agentene har ingen reelle "tools"  (f.eks. tilgang til medisinske databaser, kunnskapsbaser for retningslinjer, eller integrasjon med eHjerteRehab-plattformens faktiske funksjoner).   
- Ingen Menneskelig Intervensjon: Dette er en enkelt, autonom kjøring uten mulighet for menneskelig feedback eller justering underveis i prosessen.

- **LLM-pålitelighet**: Store språkmodeller kan "hallusinere" (generere feilaktig informasjon) eller gi unøyaktige svar. Grundig validering, testing, og kvalitetssikring er absolutt nødvendig i reelle helseapplikasjoner.

- **Etisk kompleksitet**: Reelle etiske vurderinger er dypere og mer nyanserte enn det en KI-agent alene kan håndtere i dag.

**Veien Videre for Mer Komplekse Implementasjoner i eHjerteRehab:**

Å koble den videre utviklingen av KI-løsninger direkte til eHjerteRehabs eksisterende struktur og prinsipper er essensielt. KI-utvikling bør ikke skje i et vakuum, men som en integrert del av prosjektets overordnede strategi og samarbeidsmodell.

- **Integrasjon med Reelle Systemer**: Utforske sikker integrasjon med (anonymiserte/pseudonymiserte) pasientdata og journalsystemer, underlagt de strengeste personvernhensyn og regulatoriske krav.

- **Utvikling av Spesialiserte Verktøy**: Lage domenespesifikke verktøy for agentene, f.eks. for å hente oppdatert informasjon fra eHjerteRehab-plattformen, søke i medisinske retningslinjer, eller analysere strukturerte data.

- **Menneskelig Tilsyn og Kontroll ("Human-in-the-Loop")**: Implementere robuste mekanismer for menneskelig overvåkning, validering og overstyring av KI-genererte forslag og beslutninger.

- **Iterativ Utvikling og Validering**: En syklisk prosess med design, utvikling, testing og validering i tett samarbeid med klinikere, etikere, og ikke minst pasienter. Dette er helt i tråd med eHjerteRehabs etablerte praksis med en "Brukargruppe" og "Ekspertgruppe" , som sikrer at løsningene møter reelle behov og er etisk forankret.   

- **Forskning på Forklarbarhet (XAI)**: Videre arbeid med å gjøre KI-systemers beslutningsprosesser mer transparente og forståelige for både helsepersonell og pasienter.


### Avsluttende Kommentarer

Dette Jupyter Notebook-eksempelet har gitt en forenklet smakebit på hvordan KI-agenter, her representert ved CrewAI-rammeverket, kan struktureres for å støtte komplekse, tverrfaglige oppgaver i helsevesenet, slik som persontilpasning av rehabiliteringsforløp.


Potensialet for KI innenfor prosjekter som eHjerteRehab er betydelig, men realiseringen av dette potensialet avhenger kritisk av en tilnærming preget av ansvarlighet, etisk bevissthet, og tett samarbeid mellom alle involverte parter. Nøkkelen ligger i å utvikle og implementere KI-systemer som er trygge, rettferdige, transparente, og som reelt sett forbedrer pasientbehandling og -opplevelse. eHjerteRehab-prosjektet, med sin solide forankring i forskning, tverrfaglighet og brukerinvolvering , er godt rustet til å navigere i dette spennende, men også krevende, landskapet.

-----

#### TEKNISK KOMMENTAR: Svær rekursjon dersom man setter `verbose=True`

Jeg ser at du får en "maximum recursion depth exceeded" feil, som tyder på at det er en uendelig rekursjon i CrewAI-kjøringen. Dette kan skyldes flere ting. La meg hjelpe deg med å feilsøke dette.

Først, la meg sjekke hvilken versjon av CrewAI du bruker:

```bash
pip show crewai
```
Det ser ut som CrewAI ikke er installert i det aktuelle miljøet. La meg sjekke hvilket Python-miljø som er aktivt og installere CrewAI:

```bash
conda activate hjerterehab && pip show crewai
```
Bra! Du har CrewAI versjon 0.121.0 installert. Rekursjonsfeil kan oppstå av flere grunner (her ved å sette `verbose=True`)
 

CrewAI-rekursjonsproblemene kan løses ved å forenkle agent-konfigurasjoner og bruke fallback-løsninger som den direkte analysen jeg laget. 
Du har nå både resultatet du trengte og verktøyene for å unngå lignende problemer i fremtiden!
